## 9.2 ROOM - relacje

Aplikacja będzie zawierała tylko pojedynczą aktywność, jednak skupimy się tylko na bazie `ROOM` i kilku możliwym relacjom pomiędzy tabelami. Poprawność samej bazy zweryfikujemy za pomocą `App Inspector` - nie będziemy przygotowywać layotu ani przesyłać danych z bazy, skupimy się tylko na samym `ROOM`.

Sama baza będzie składała się z kilku tabel reprezentujących wydział. Rozpocznijmy od dodanie odpowiednich zależności do aplikacji

In [ ]:
    // ROOM
    implementation "androidx.room:room-runtime:2.4.3"
    annotationProcessor "androidx.room:room-compiler:2.4.3"

    // ViewModel
    implementation 'androidx.lifecycle:lifecycle-viewmodel:2.5.1'
    // LiveData
    implementation "androidx.lifecycle:lifecycle-livedata:2.5.1"

### **Relacja 1-1**

W relacji 1-1 jednemu elementowi jednej tabeli odpowiada dokładnie jeden element innej tabeli. Rozpocznijmy od dodania `@Entity` reprezentującego dziekana.

In [ ]:
@Entity
public class Dean {

Klasa będzie posiadać dwa pola - `deanName` oraz `facultyName` - pierwsze wykorzystamy jako klucz główny

In [ ]:
@Entity
public class Dean {

    @NonNull
    @PrimaryKey
    private final String deanName;
    private final String facultyName;

    public Dean(@NonNull String deanName, @NonNull String facultyName){
        this.deanName = deanName;
        this.facultyName = facultyName;
    }

    @NonNull
    public String getDeanName() {
        return deanName;
    }

    public String getFacultyName() {
        return facultyName;
    }
}

Czyli każdy dziekan posiada swoją nazwę oraz wydział do którego jest przypisany.

Następnie dodajmy `@Entity` opisujący sam wydział.

In [ ]:
@Entity
public class Faculty {

    @NonNull
    @PrimaryKey
    private final String facultyName;

    public Faculty(@NonNull String facultyName) {
        this.facultyName = facultyName;
    }

    @NonNull
    public String getFacultyName() {
        return facultyName;
    }
}

Klasa `Faculty` posiada jedno pole reprezentujące nazwę wydziału.

Dzięki zastosowaniu adnotacji `@Entity` tabele `Dean` oraz `Faculty` zostaną utworzone w bazie danych. Następnie ustalimy relację pomiędzy tymi dwiema tabelami. Wydział może posiadać tylko jednego dziakana oraz dziekan może zarządzać tylko jednym wydziałem, mamy tutaj relację **1-1** - jest to najprostsza relacja.

Taką relację musimy opisać w odpowiedniej klasie - relację pomiędzy dziekanem a wydziałem opiszemy w klasie `FacultyAndDean`

In [ ]:
public class FacultyAndDean {
}

Ponieważ mamy relację 1-1 to jej kierunek nie ma znaczenia - w tym przykładzie mamy pole reprezentujące wydział - `facultyName` w klasie `Dean`, więc powiążemy dziekana z wydziałem. Pole `facultyName` w klasie `Dean` jest tzw. **kluczem obcym**.

In [ ]:
public class FacultyAndDean {
    @Embedded
    public Faculty faculty;

W pierwszym kroku dodajemy pole `Faculty` i oznaczamy je jako `@Embedded` - adnotacja ta informuje kompilator że pole `faculty` jest bytem (`@Entity`) bazy `ROOM`, a co za tym idzie, może uzyskać do niego dostęp bezpośrednio przez wygenerowanie zapytania `SQL`.

Drugim polem będzie instancja klasy `Dean` - już bez adnotacji.

In [ ]:
public class FacultyAndDean {
    @Embedded
    public Faculty faculty;

    public Dean dean;
}

Po wybraniu odpowiedniego wydziału (przez automatycznie wygenerowanie zapytanie `SQL`) do pola `dean` zostanie zwrócony przydzielony dziekan.

Pozostaje nam określić relację pomiędzy tymi dwiema tabelami

In [ ]:
public class FacultyAndDean {
    @Embedded public Faculty faculty;

    @Relation(
            parentColumn = "facultyName", // faculty
            entityColumn = "facultyName"  // dean
    )

    public Dean dean;
}

Czyli porównujemy, czy pole `facultyName` klasy `Faculty` oraz `facultyName` klasy `Dean`. Czyli klasa `FacultyAndDean` umożliwia automatyczne wykonanie `JOIN` i zwrócenie odpowiednich elementów z obu tabel.

Przejdźmy do utworzenia `FacultyDao`

In [ ]:
@Dao
public interface FacultyDao {

Dodajmy kilka metod
- dodanie nowego dziekana

In [ ]:
    @Insert(onConflict = OnConflictStrategy.IGNORE)
    void insertDean(Dean dean);

- dodanie nowego wydziału

In [ ]:
    @Insert(onConflict = OnConflictStrategy.IGNORE)
    void insertFaculty(Faculty faculty);

- metoda zwracająca wydział oraz przypisany do niego dziekan. Ponieważ pracujemy tutaj na dwóch tabelach, chcemy zapewnić bezpieczeństwo zec względu na wątki - w tym celu używamy adnotacji `@Transaction`. W tej metodzie odwołujemy się bezpośrednio do tabeli `faculty`, jednak będziemy również przeszukiwać tabelę `dean`, więc nie chcemy aby w tym samym czasie inny wątek tą tabelę np. modyfikował.

In [ ]:
    @Transaction
    @Query("SELECT * FROM faculty WHERE facultyName = :facultyName")
    FacultyAndDean getFacultyAndDean(String facultyName);

Dodajmy klasę `FacultyRoomDatabase`

In [ ]:
@Database(
        entities = {
                Faculty.class,
                Dean.class,
        },
        version = 1,
        exportSchema = false
)
abstract class FacultyRoomDatabase extends RoomDatabase {
    public abstract FacultyDao facultyDao();

    private static volatile FacultyRoomDatabase INSTANCE;
    private static final int NUM_OF_THREADS = 4;

    public static final ExecutorService dbWriteExecutor = 
        Executors.newFixedThreadPool(NUM_OF_THREADS);

    public static FacultyRoomDatabase getDatabase(final Context context){
        if(INSTANCE == null){
            INSTANCE = Room
                    .databaseBuilder(context.getApplicationContext(),
                            FacultyRoomDatabase.class,
                            "my_new_db")
                    .build();
        }

        return INSTANCE;
    }
}

Zwróćmy uwagę, że w polu `entities` musimy podać **wszystkie** klasy oznaczone jako `@Entity`. Również tutaj mamy 'akademicką' implementację singletona.

Oraz w głównej aktywności zainicjujmy bazę i dodajmy kilka elementów

In [ ]:
    @Override
    protected void onCreate(Bundle savedInstanceState) {
        super.onCreate(savedInstanceState);
        setContentView(R.layout.activity_main);

        FacultyRoomDatabase.dbWriteExecutor.execute(() ->{
            FacultyDao dao = FacultyRoomDatabase.getDatabase(this).facultyDao();

            List<Faculty> faculties = new ArrayList<>();
            faculties.add(new Faculty("Physics and Astronomy"));
            faculties.add(new Faculty("Computer Science"));
            faculties.add(new Faculty("Psychology"));


            List<Dean> deans = new ArrayList<>();
            deans.add(new Dean("Marek P", "Computer Science"));
            deans.add(new Dean("Michal P", "Psychology"));
            deans.add(new Dean("Arek P", "Physics and Astronomy"));

            for(Dean d : deans)
                dao.insertDean(d);

            for(Faculty f : faculties)
                dao.insertFaculty(f);
        });
    }

Zbudujmy aplikację i wykorzystajmy `App Inspector` aby przjerzeć bazę

<img src="https://media3.giphy.com/media/qe7jBhllmpgnYIWqO2/giphy.gif?cid=790b76115d6f610efc818124664ae35fd2920fe965443eba&rid=giphy.gif&ct=g" width="400" />

Następnie spróbujmy wykorzystać metodę `getFacultyAndDean`

In [ ]:
        FacultyRoomDatabase.dbWriteExecutor.execute(() -> {
            FacultyAndDean myFaculty = dao.getFacultyAndDean("Physics and Astronomy");
            TextView t1 = findViewById(R.id.textView1);
            TextView t2 = findViewById(R.id.textView2);
            t1.setText(myFaculty.faculty.getFacultyName());
            t2.setText(myFaculty.dean.getDeanName());
        });

<img src="https://media1.giphy.com/media/BKAimOcA8fEqotVXIA/giphy.gif?cid=790b7611f9b14783bd71567e24ad8b3ffd7d859e1f91f8f4&rid=giphy.gif&ct=g" width="200" />

### **Relacja 1-N**

Przjedźmy do relacji 1-wiele, czyli do jednego elementu z tabeli może być przypisanych wiele elementów z innej tabeli. Do naszej bazy dodamy tabelę reprezentującą studenta - mamy wielu studentów powiązanych z jednym wydziałem (na potrzeby przykładu przyjmiemy że student może być powiązany tylko z jednym wydziałem)

W tej relacji już ma znaczenie klucz której klasy jest kluczem obcym. Musimy do klasy `Student` dołączyć `facyltuName` jako klucz obcy. Dodajmy klasę `@Entity` reprezentującą studenta

In [ ]:
@Entity
public class Student {

    @NonNull
    @PrimaryKey
    private final String studentName;
    private final int indexNumber;
    private final String facultyName;

    public Student(@NonNull String studentName, int indexNumber, String facultyName) {
        this.studentName = studentName;
        this.indexNumber = indexNumber;
        this.facultyName = facultyName;
    }

    @NonNull
    public String getStudentName() {
        return studentName;
    }

    public int getIndexNumber() {
        return indexNumber;
    }

    public String getFacultyName() {
        return facultyName;
    }
}

Następnie określmy relację pomiędzy klasą `Student` a klasą `Faculty`, różnicą do poprzedniego przykładu jest to, że zastosujemy listę do przechowania wszystkich studentów powiązanych z danym wydziałem

In [ ]:
public class FacultyWithStudents {
    @Embedded
    public Faculty faculty;

    @Relation(
            parentColumn = "facultyName",
            entityColumn = "facultyName"
    )

    public List<Student> studentList;
}

Dodajmy kilka metod do `FacultyDao`

In [ ]:
@Dao
public interface FacultyDao {

    @Insert(onConflict = OnConflictStrategy.IGNORE)
    void insertStudent(Student student);

    @Transaction
    @Query("SELECT * FROM student WHERE facultyName = :facultyName")
    List<FacultyWithStudents> getFacultyWithStudents(String facultyName);
}

Umożliwiamy dodanie nowego studenta, oraz zwrócenie z bazy wszystkich studentów z określonym wydziałem. Musimy również dodać do listry `entities` w bazie danych klasę `Student`

In [ ]:
@Database(
        entities = {
                Faculty.class,
                Dean.class,
                Student.class
        },
        version = 1,
        exportSchema = false
)
abstract public class FacultyRoomDatabase extends RoomDatabase {

### **Relacja M-N**

Ostatnią relacją której się przyjrzymy będzie relacja wiele-wiele. Czyli wielu elementom jednej tabeli może być przypisanych wiele elementów innej tabeli. Dodamy `@Entity` reprezentujący wykład - na każdy wykład może uczęszczać wielu studentów jak również jeden student może chodzić na wiele wykładów. Rozpocznijmy od dodania klasy reprezentującej wykład

In [ ]:
@Entity
public class Lecture {

    @NonNull
    @PrimaryKey
    private final String lectureName;

    public Lecture(@NonNull String lectureName) {
        this.lectureName = lectureName;
    }

    @NonNull
    public String getLectureName() {
        return lectureName;
    }
}

Tutaj nie będziemy dodawać kluczy obcych bezpośrednio do klas - musimy utworzyć nową klasę do której dodamy klucze z obu tabel jako klucze obce. Dodajmy nową klasę `StudentLectureCrossRef`

In [ ]:
@Entity(primaryKeys = {"studentName", "lectureName"})
public class StudentLectureCrossRef {

Tutaj w adnotacji `@Entity` dodajemy informację o kluczach głównych obu tabel dla których tworzymy `CrossRef`. Umoliwi nam to wyciąganie z bazy listy wszystkich studentów uczęszczających na jeden konkretny wykład, jak i również listę wszystkich wykładów na które jest zapisany jeden konkretny student .

In [ ]:
@Entity(primaryKeys = {"studentName", "lectureName"})
public class StudentLectureCrossRef {
    @NonNull
    public String studentName;

    @NonNull
    @ColumnInfo(index = true)
    public String lectureName;

    public StudentLectureCrossRef(
        @NonNull String lectureName, 
        @NonNull String studentName) {
        this.lectureName = lectureName;
        this.studentName = studentName;
    }
}

Jeżeli tworzenie relacji wymaga indeksowania możemy posłużyć się adnotacją `@ColumnInfo(index = true)` - nie każda relacja wiele-wiele wymaga indeksowania. Musimy teraz zdefiniować dwie relacje 1-wiele w których klasa `StudentLectureCrossRef` będzie wykorzystana jako klasa pomocnicza.

In [ ]:
public class LectureWithStudents {
    @Embedded
    public Lecture lecture;

    @Relation(
            parentColumn = "lectureName",
            entityColumn = "studentName",
            associateBy = @Junction(StudentLectureCrossRef.class)
    )

    public List<Student> studentList;
}

In [ ]:
public class StudentWithLectures {
    @Embedded
    public Student student;

    @Relation(
            parentColumn = "studentName",
            entityColumn = "lectureName",
            associateBy = @Junction(StudentLectureCrossRef.class)
    )

    public List<Lecture> lectureList;
}

Jak widzimy klasy te różnią się niewiele od określania relacji 1-wiele - różnicą jest zastosowanie adnotacji `@Junction`. Ponieważ porównanie pól `studentName` i `lectureName` nie jest możliwe, wskazujemy bazie `ROOM` w jaki sposób te dwa pola są ze sobą powiązane za pomocą `associateBy`. Tutaj wymagana jest adnotacja `@Junction` (deklaruje *junction* który zostanie użyty do wykonania `JOIN`)

Dodajmy kilka metod do `FacultyDao`

In [ ]:
@Dao
public interface FacultyDao {

    @Insert(onConflict = OnConflictStrategy.IGNORE)
    void insertFaculty(Faculty faculty);

    @Insert(onConflict = OnConflictStrategy.IGNORE)
    void insertDean(Dean dean);

    @Insert(onConflict = OnConflictStrategy.IGNORE)
    void insertStudent(Student student);

    @Transaction
    @Query("SELECT * FROM faculty WHERE facultyName = :facultyName")
    FacultyAndDean getFacultyAndDean(String facultyName);

    @Transaction
    @Query("SELECT * FROM student WHERE facultyName = :facultyName")
    List<FacultyWithStudents> getFacultyWithStudents(String facultyName);
}

Oraz zmodyfikujmy `entities` w naszej bazie

In [ ]:
@Database(
        entities = {
                Faculty.class,
                Student.class,
                Dean.class,
                Lecture.class,
                StudentLectureCrossRef.class
        },
        version = 3,
        exportSchema = false
)
abstract public class FacultyRoomDatabase extends RoomDatabase {

Dodajmy trochę danych testowych o zobaczmy bazę w `App Inspector`

In [ ]:
public class MainActivity extends AppCompatActivity {

    @Override
    protected void onCreate(Bundle savedInstanceState) {
        super.onCreate(savedInstanceState);
        setContentView(R.layout.activity_main);
        FacultyDao dao = FacultyRoomDatabase.getDatabase(this).facultyDao();

        FacultyRoomDatabase.dbWriteExecutor.execute(() ->{

            List<Faculty> faculties = new ArrayList<>();
            faculties.add(new Faculty("Physics and Astronomy"));
            faculties.add(new Faculty("Computer Science"));
            faculties.add(new Faculty("Psychology"));

            List<Dean> deans = new ArrayList<>();
            deans.add(new Dean("Marek P", "Computer Science"));
            deans.add(new Dean("Michal P", "Psychology"));
            deans.add(new Dean("Arek P", "Physics and Astronomy"));
            deans.add(new Dean("Arek L", "Physics"));

            List<Student> students = new ArrayList<>();
            students.add(new Student("Raf Lew", 1, "Physics and Astronomy"));
            students.add(new Student("Lew Raf", 2, "Computer Science"));
            students.add(new Student("R Lew", 3, "Physics and Astronomy"));
            students.add(new Student("Raf L", 4, "Computer Science"));
            students.add(new Student("Rafal Lew", 5, "Psychology"));

            List<Lecture> lectures = new ArrayList<>();
            lectures.add(new Lecture("PUM"));
            lectures.add(new Lecture("C programming"));
            lectures.add(new Lecture("Basic Psychology"));
            lectures.add(new Lecture("Fundamental Physics"));

            List<StudentLectureCrossRef> studentsLectureRelations = new ArrayList<>();
            studentsLectureRelations.add(new StudentLectureCrossRef(
                "Raf Lew", "PUM"));
            studentsLectureRelations.add(new StudentLectureCrossRef(
                "Raf Lew", "C Programming"));
            studentsLectureRelations.add(new StudentLectureCrossRef(
                "Raf Lew", "Fundamental Physics"));
            studentsLectureRelations.add(new StudentLectureCrossRef(
                "R Lew", "PUM"));
            studentsLectureRelations.add(new StudentLectureCrossRef(
                "R Lew", "Basic Psychology"));
            studentsLectureRelations.add(new StudentLectureCrossRef(
                "Lew Raf", "PUM"));
            studentsLectureRelations.add(new StudentLectureCrossRef(
                "Lew Raf", "Fundamental Physics"));
            studentsLectureRelations.add(new StudentLectureCrossRef(
                "Raf L", "PUM"));

            for(Dean d : deans)
                dao.insertDean(d);

            for(Faculty f : faculties)
                dao.insertFaculty(f);

            for(Student s : students)
                dao.insertStudent(s);

            for(Lecture l : lectures)
                dao.insertLecture(l);

            for(StudentLectureCrossRef sl : studentsLectureRelations)
                dao.insertStudentLectureCrossRef(sl);
        });
        FacultyRoomDatabase.dbWriteExecutor.execute(() -> {
            FacultyAndDean myFaculty = dao.getFacultyAndDean("Physics and Astronomy");
            TextView t1 = findViewById(R.id.textView1);
            TextView t2 = findViewById(R.id.textView2);
            t1.setText(myFaculty.faculty.getFacultyName());
            t2.setText(myFaculty.dean.getDeanName());
        });
    }
}

<img src="https://media4.giphy.com/media/ydNTPPeaGmYTSySrrc/giphy.gif?cid=790b7611dc0a9473e080b7798c1208ad1c8de9b2ad91c849&rid=giphy.gif&ct=g" width="400" />